![HydroSAR Banner](./NotebookAddOns/HydroSARbanner.jpg)

# Flood Depth Estimation with Flood Extent Maps

## Part of NASA A.37 Project: Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters

### PI:Franz J. Meyer
**Version 0.1.8 - 2021/01/24**

Change Log: See bottom of the notebook.
    
**Batuhan Osmanoglu, MinJeong Jo; NASA Goddard Space Fligth Center**

This notebook provides the processor to generate Flood Depth map using the product generated by **Hyp3 Change Detection-Threshold** processor. This notebook can be used to generate **Multiple** FD Products

*Note: Before you start to use the notebook, Hyp3-generated change detection maps in Geotiff format need to be placed in your own data folder. For the HydroSAR Training, these SAR data are already available to you after completion of Lab 2.*

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/hydrosar':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "hydrosar" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "hydrosar" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "hydrosar" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## Importing Relevant Python Packages

In [ ]:
#Setup Environment
from pathlib import Path
import urllib
import pprint
import warnings

import numpy as np
from osgeo import gdal
gdal.UseExceptions()
from osgeo import osr
import pylab as pl
from scipy import ndimage
from scipy import optimize
from scipy import stats
import astropy
import astropy.convolution
import pykrige
import pysheds
from pysheds.grid import Grid
from affine import Affine
import rasterio
import pyproj

from ipyfilechooser import FileChooser

# #Download packages
codes_folder = Path('/home/jovyan/adore_doris')
if not codes_folder.exists():
    !git clone https://github.com/bosmanoglu/adore-doris.git {codes_folder}
else:
    print("here")
    !git --git-dir={codes_folder/".git"} pull origin master

import sys

if str(codes_folder) not in sys.path:
    sys.path.append(f'{codes_folder}/lib/python')
    sys.path.append(codes_folder)
    
# #import modules after downloads
import gis
from tqdm.notebook import tqdm

# Define Convenience Functions

In [ ]:
from os import system

# Define convenience functions
def bounding_box_inside_bounding_box(small, big):
    s0 = np.array([p[0] for p in small])
    s1 = np.array([p[1] for p in small])
    b0 = np.array([p[0] for p in big])
    b1 = np.array([p[1] for p in big])
    inside = True
    if s0.min() < b0.min():
        inside = False
    if s0.max() > b0.max():
        inside = False
    if s1.min() < b1.min():
        inside = False
    if s1.max() > b1.max():
        inside = False
    return inside


def getGeoTransform(filename):
    warnings.warn("getGeoTransform will be deprecated in the future. Please use read_data instead.", PendingDeprecationWarning)
    return get_geotransform(filename)


def get_geotransform(filename):
    '''
    [top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution]=getGeoTransform('/path/to/file')
    '''
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    ds = gdal.Open(filename)
    return ds.GetGeoTransform()
    
    
def build_vrt(filename, input_file_list):
    resolution = gdal.Info(input_file_list[0], format='json')['geoTransform'][1]
    vrt_options = gdal.BuildVRTOptions(resampleAlg='near', 
                                       separate=False,
                                       xRes=resolution,
                                       yRes=resolution,
                                       targetAlignedPixels=True)
    gdal.BuildVRT(filename, input_file_list, options=vrt_options)

    
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths


def gdal_get_projection(filename, out_format='proj4'):
    """
    epsg_string=get_epsg(filename, out_format='proj4')
    """
    try:
        ds = gdal.Open(filename, gdal.GA_ReadOnly)
        srs = gdal.osr.SpatialReference()
        srs.ImportFromWkt(ds.GetProjectionRef())
    except: #I am not sure if this is working for datasets without a layer. The first try block should work mostly.
        ds = gdal.Open(filename, gdal.GA_ReadOnly)
        ly = ds.GetLayer()
        if ly is None:
            print(f"Can not read projection from file:{filename}")
            return None
        else:
            srs = ly.GetSpatialRef()
    if out_format.lower() == 'proj4':
        return srs.ExportToProj4()
    elif out_format.lower() == 'wkt':
        return srs.ExportToWkt()
    elif out_format.lower() == 'epsg':
        crs = pyproj.crs.CRS.from_proj4(srs.ExportToProj4())
        return crs.to_epsg()


def get_size(filename):
    """(width, height) = get_size(filename)
    """
    ds = gdal.Open(filename)
    width = ds.RasterXSize
    height = ds.RasterYSize
    ds = None
    return (width, height)


def get_proj4(filename):
    f = rasterio.open(filename)
    return pyproj.Proj(f.crs, preserve_units=True)  #used in pysheds


def clip_gT(gT, xmin, xmax, ymin, ymax, method='image'):
    '''calculate new geotransform for a clipped raster either using pixels or projected coordinates.
    clipped_gT=clip_gT(gT, xmin, xmax, ymin, ymax, method='image')
    method: 'image' | 'coord'
    '''
    if method == 'image':
        y, x = xy2coord(ymin, xmin, gT); #top left, reference, coordinate
    if method == 'coord':
      #find nearest pixel
      yi, xi = coord2xy(ymin, xmin, gT)
      #get pixel coordinate
      y, x = xy2coord(yi, xi, gT)
    gTc = list(gT)
    gTc[0] = y
    gTc[3] = x
    return tuple(gTc)


def xy2coord(x, y, gT):
    '''
    lon,lat=xy2coord(x,y,geoTransform)
    projects pixel index to position based on geotransform.
    '''
    coord_x = gT[0] + x*gT[1] + y*gT[2]
    coord_y = gT[3] + x*gT[4] + y*gT[5]
    return coord_x, coord_y


def coord2xy(x, y, gT):
    '''
    x,y = coord2xy(lon, lat, geoTransform)
    calculates pixel index closest to the lon, lat.
    '''
    #ref: https://gis.stackexchange.com/questions/221292/retrieve-pixel-value-with-geographic-coordinate-as-input-with-gdal/221430
    xOrigin = gT[0]
    yOrigin = gT[3]
    pixelWidth = gT[1]
    pixelHeight = -gT[5]

    col = np.array((x - xOrigin) / pixelWidth).astype(int)
    row = np.array((yOrigin - y) / pixelHeight).astype(int)

    return row, col


def fitSurface(x, y, z, X, Y):
    p0 = [0, 0.1, 0.1]
    fitfunc = lambda p, x, y: p[0] + p[1] * x + p[2] * y
    errfunc = lambda p, x, y, z: abs(fitfunc(p,x,y) - z)
    planefit, success = optimize.leastsq(errfunc, p0, args=(x,y,z))
    return fitfunc(planefit, X, Y)    


def nonan(a, rows=False):
    if rows:
        return a[np.isnan(a).sum(1)==0]
    else:
        return a[~np.isnan(a)]

    
def get_wesn(filename, t_srs=None):
    bb = bounding_box(filename, t_srs=t_srs)
    w = np.inf
    e = -np.inf
    n = -np.inf
    s = np.inf
    for p in bb:
        if p[0] < w:
            w = p[0]
        if p[0] > e:
            e = p[0]
        if p[1] < s:
            s = p[1]
        if p[1] > n:
            n = p[1]
    return [w, e, s, n]


def bounding_box(filename, t_srs=None):
    """
    ((lon1,lat1), (lon2,lat2), (lon3,lat3), (lon4,lat4))=bounding_box('/path/to/file', t_srs=None) #returns x,y in native coordinate system
    ((lon1,lat1), (lon2,lat2), (lon3,lat3), (lon4,lat4))=bounding_box('/path/to/file', t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    """
    gT = getGeoTransform(filename)
    width, height = get_size(filename)
    pts = (xy2coord(0, 0, gT), xy2coord(width, 0, gT), xy2coord(width, height, gT), xy2coord(0, height, gT))
    if t_srs is None:
        return pts
    else:
        pts_tsrs = []
        s_srs = gdal_get_projection(filename, out_format='proj4')
        for p in pts:
            pts_tsrs.append(transform_point(p[0], p[1], 0, s_srs=s_srs, t_srs=t_srs))
    return tuple(pts_tsrs)   


def transform_point(x, y, z, 
                    s_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', 
                    t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'):
    '''
    transform_point(x,y,z,s_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    
    Known Bugs: gdal transform may fail if a proj4 string can not be found for the EPSG or WKT formats. 
    '''    
    srs_cs = osr.SpatialReference()    
    if "EPSG" == s_srs[0: 4]:    
        srs_cs.ImportFromEPSG(int(s_srs.split(':')[1]));
    elif "GEOCCS" == s_srs[0:6]:
        srs_cs.ImportFromWkt(s_srs);
    else:
        srs_cs.ImportFromProj4(s_srs);

    trs_cs = osr.SpatialReference()    
    if "EPSG" == t_srs[0:4]:    
        trs_cs.ImportFromEPSG(int(t_srs.split(':')[1]));
    elif "GEOCCS" == t_srs[0:6]:
        trs_cs.ImportFromWkt(t_srs);
    else:
        trs_cs.ImportFromProj4(t_srs);
    if int(gdal.VersionInfo()) > 2999999: #3010300
        #https://gdal.org/tutorials/osr_api_tut.html#crs-and-axis-order
        # https://github.com/OSGeo/gdal/blob/master/gdal/MIGRATION_GUIDE.TXT
        srs_cs.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)
        trs_cs.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)
    transform = osr.CoordinateTransformation(srs_cs, trs_cs) 

    if numel(x) > 1:
        return [transformPoint(x[k], y[k], z[k]) for k in range(numel(x))]
    else:
        try:
            return transform.TransformPoint((x, y, z));
        except: 
            return transform.TransformPoint(x, y, z)

def get_waterbody(filename, ths):
    corners = bounding_box(filename)

    epsg = gdal_get_projection(filename, out_format='epsg')
    if epsg == "4326":         
        corners = bounding_box(filename)
    else:
        srs = gdal_get_projection(filename, out_format='proj4')
        corners = bounding_box(filename, t_srs="EPSG:4326")
    west = corners[0][0]
    east = corners[1][0]
    south = corners[2][1]
    north = corners[0][1]    
        
# S_WATER directory is being created above working directory (i.e. Bangladesh, El_Salvador, etc.) to avoid clutter.
# Modify here with "work_path" if you want S_WATER directory generated within your working directory.
    cwd = Path.cwd()
    sw_path = cwd/f"S_WATER"

    if not sw_path.exists():
        sw_path.mkdir()

    lon = np.floor(west/10)
    lon = int(abs(lon*10))
    lat = np.ceil(north/10)
    lat = int(abs(lat*10))

    if (west < 0 and north < 0):
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}W_{lat}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat}S.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon-10}W_{lat}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat}S.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}W_{lat+10}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat+10}S.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon-10}W_{lat+10}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat+10}S.tif")
        print(f"lon: {lon}-{lon-10}W, lat: {lat}-{lat+10}S ")

    elif (west < 0 and north >= 0):
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}W_{lat}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat}N.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon-10}W_{lat}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat}N.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}W_{lat-10}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat-10}N.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon-10}W_{lat-10}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat-10}N.tif")
        print(f"lon: {lon}-{lon-10}W, lat: {lat}-{lat-10}N ")


    elif (west >= 0 and north < 0):
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}E_{lat}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat}S.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon+10}E_{lat}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat}S.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}E_{lat+10}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat+10}S.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon+10}E_{lat+10}Sv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat+10}S.tif")
        print(f"lon: {lon}-{lon+10}E, lat: {lat}-{lat+10}S ")

    else:
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}E_{lat}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat}N.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon+10}E_{lat}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat}N.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon}E_{lat-10}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat-10}N.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2019v2/occurrence/occurrence_{lon+10}E_{lat-10}Nv1_1_2019.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat-10}N.tif")
        print(f"lon: {lon}-{lon+10}E, lat: {lat}-{lat-10}N ")

    # Building the virtual raster for Change Detection product(tiff)
    product_wpath = cwd/f"S_WATER/surface_water*.tif"

    #wildcard_path is not being used for now
    #wildcard_path = f"{cwd}/change_VV_20170818T122205_20170830T122203.tif"
    print(product_wpath)

    get_ipython().system(f'gdalbuildvrt {product_wpath.parent}/surface_water_map.vrt $product_wpath')


    #Clipping/Resampling Surface Water Map for AOI
    dim = get_size(filename)
    if epsg == "4326":
        cmd_resamp = f"gdalwarp -overwrite -te {west} {south} {east} {north} -ts {dim[0]} {dim[1]} -r lanczos {product_wpath.parent}/surface_water_map.vrt {product_wpath.parent}/surface_water_map_clip.tif"
    else:   
        corners=bounding_box(filename) # we now need corners in the non EPSG:4326 format.  
        west = corners[0][0]
        east = corners[1][0]
        south = corners[2][1]
        north = corners[0][1]            
        cmd_resamp = f"gdalwarp -overwrite -t_srs '{srs}' -te {west} {south} {east} {north} -ts {dim[0]} {dim[1]} -r nearest {product_wpath.parent}/surface_water_map.vrt {product_wpath.parent}/surface_water_map_clip.tif"        
    print(cmd_resamp)
    system(cmd_resamp)

    #load resampled water map
    wimage_file = f"{product_wpath.parent}/surface_water_map_clip.tif"
    water_map = gdal.Open(wimage_file)
    
    swater_map = gis.readData(wimage_file)
    wmask= swater_map > ths   #higher than 30% possibility (present water)
        
    return wmask


def numel(x):
    if isinstance(x, int):
      return 1
    elif isinstance(x, np.double):
      return 1
    elif isinstance(x, float):
      return 1
    elif isinstance(x, str):
      return 1
    elif isinstance(x, list) or isinstance(x, tuple):
      return len(x)
    elif isinstance(x, np.ndarray):
      return x.size
    else: 
      print('Unknown type {}.'.format(type(x)))
      return None

    
def yesno(yes_no_question = "[y/n]"):
    while True:
        # raw_input returns the empty string for "enter"
        yes = {'yes','y', 'ye'}
        no = {'no','n'}

        choice = input(yes_no_question+"[y/n]").lower()
        if choice in yes:
            return True
        elif choice in no:
            return False
        else:
            print("Please respond with 'yes' or 'no'")
            
            
def fill_nan(arr):
    """
    filled_arr=fill_nan(arr)
    Fills Not-a-number values in arr using astropy. 
    """    
    kernel = astropy.convolution.Gaussian2DKernel(x_stddev=3) #kernel x_size=8*stddev
    arr_type = arr.dtype          
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        while np.any(np.isnan(arr)):
            arr = astropy.convolution.interpolate_replace_nans(arr.astype(float), kernel, convolve=astropy.convolution.convolve)
    return arr.astype(arr_type) 


def logstat(data, func=np.nanstd):
    """ stat=logstat(data, func=np.nanstd)
       calculates the statistic after taking the log and returns the statistic in linear scale.
       INF values inside the data array are set to nan. 
       The func has to be able to handle nan values. 
    """
    ld = np.log(data)
    ld[np.isinf(ld)] = np.nan #requires func to handle nan-data.
    st = func(ld)
    return np.exp(st)

def iterative(hand, extent, water_levels=range(15)):
    #accuracy=np.zeros(len(water_levels))    
    #for k,w in enumerate(water_levels):
    #    iterative_flood_extent=hand<w
    #    TP=np.nansum(np.logical_and(iterative_flood_extent==1, extent==1)) #true positive
    #    TN=np.nansum(np.logical_and(iterative_flood_extent==0, extent==0)) # True negative
    #    FP=np.nansum(np.logical_and(iterative_flood_extent==1, extent==0)) # False positive
    #    FN=np.nansum(np.logical_and(iterative_flood_extent==0, extent==1)) # False negative
    #    #accuracy[k]=(TP+TN)/(TP+TN+FP+FN) #accuracy 
    #    accuracy[k]=TP/(TP+FP+FN) #threat score
    #best_water_level=water_levels[np.argmax(accuracy)]
    
    def _goal_ts(w):
        iterative_flood_extent=hand<w # w=water level
        TP = np.nansum(np.logical_and(iterative_flood_extent==1, extent==1)) #true positive
        TN = np.nansum(np.logical_and(iterative_flood_extent==0, extent==0)) # True negative
        FP = np.nansum(np.logical_and(iterative_flood_extent==1, extent==0)) # False positive
        FN = np.nansum(np.logical_and(iterative_flood_extent==0, extent==1)) # False negative
        return 1 - TP / (TP + FP + FN) #threat score #we will minimize goal func, hence 1-threat_score.
    #bounds=(min(water_levels), max(water_levels))
    #opt_res=optimize.minimize(_goal_ts, max(bounds),method='TNC',bounds=[bounds],options={'xtol':0.1, 'scale':1})

    class MyBounds(object):
        def __init__(self, xmax=[max(water_levels)], xmin=[min(water_levels)]):
            self.xmax = np.array(xmax)
            self.xmin = np.array(xmin)
        def __call__(self, **kwargs):
            x = kwargs["x_new"]
            tmax = bool(np.all(x <= self.xmax))
            tmin = bool(np.all(x >= self.xmin))
            return tmax and tmin
    mybounds = MyBounds()
    x0 = [np.mean(water_levels)]
    opt_res = optimize.basinhopping(_goal_ts, x0, niter=10000, niter_success=100, accept_test=mybounds)
    if opt_res.message[0] == 'success condition satisfied' or opt_res.message[0] == 'requested number of basinhopping iterations completed successfully':
        best_water_level = opt_res.x[0]
    else:        
        best_water_level = np.inf # set to inf to mark unstable solution.    
    return best_water_level

# Define Some Common Parameters

This section allows you to customize how flood depth estimation is performed. **The main paramters that users might want to change are:**
    
- **Input File Naming Scheme:** This is only relevant if you are interested in mosaicking large areas. This gives you the option of either picking initial flood mapping information created in Lab 2 (naming scheme */*_water_mask_combined.tiff*) or final post-processed flood mapping information (naming scheme */*_fcWM.tiff*) for flood depth calculation [**for the HydroSAR training, please do not change this variable from its default**]
- **Estimator:** Three different estimation approaches were implemented and are currently being tested by the HydroSAR team: 
- **Iterative:** Basin hopping optimization method to match flooded areas to flood depth estimates given the HAND layer. From our current experience, this is the most accurate, but also the most time consuming approach.
- **Normalized Median Absolute Deviation (nmad):** Uses a median operator to estimate the variation to increase robustness in the presence of outliers. [**We will use this approach for the HydroSAR training**].
- **Logstat:** This approach calculates mean and standard deviation of HAND heights in the logarithmic domain to improve robustness for very non-Gaussian data distributions.
- **Numpy:** Calculates statistics needed in the approach in linear scale. This approach is least robust to outliers and non-Gaussian distributions.

In [ ]:
#parameters setup
version = "0.1.8"
water_classes = [1, 2, 3, 4, 5] # 1 has to be a water class, 0 is no water Others are optional.
pattern = "*_water_mask_combined.tiff" #"filter_*_amp_Classified.tif"
show_plots = True #turn this off for debugging with IPDB
water_level_sigma = 3 #use 3*std to estimate max. water height (water level) for each object. Used for numpy, nmad,logstat
estimator = "nmad" # iterative, numpy, nmad or logstat
iterative_bounds = [0, 15] #only used for iterative
output_prefix = '' # Output file is created in the same folder as flood extent. A prefix can be added to the filename.
known_water_threshold = 30 #Threshold for extracting the known water area in percent. 
if show_plots:
    %matplotlib widget

# Prepare Data Set for HAND Calculation

**Enter the path to the directory holding your tiffs:** Here we ask you if you want to calculate flood depth across a mosaic or for a single file. [**Please select "single file" for the HydroSAR Training Exercise**].

In [ ]:
#Here we ask if we are processing a spatial mosaic or a single file. 
if yesno("Would you like to mosaic multiple files (e.g. large area coverage from multiple scenes)?"):
    print(f"Choose one of the water extent files inside the folder. All files with matching the following will be processed: {pattern}")
    file_folder_func = lambda x: Path(x).parent
    single_file = False
else:    
    print("Choose your GDAL compatible Classified water extent file using the file browser below:")
    file_folder_func = lambda x: x
    single_file = True
f = FileChooser(Path.cwd())
display(f)

In [ ]:
from os import WEXITSTATUS

work_path = Path(file_folder_func(f.selected)).parent

#Check if folder or file
tiff_path = Path(file_folder_func(f.selected))

if not single_file:
    #merge all tifs
    tiffs = list(map(str,list(tiff_path.rglob(f"{pattern}"))))
    print("Processing the following files:")
    pprint.pprint(tiffs)
    
# May need to change location of where Water_Masks.vrt is being created
    combined_vrt = tiff_path/f'{tiff_path.name}.vrt'
    combined_tif = tiff_path/f'{tiff_path.name}.tif'
    
    print(combined_vrt)
    print(combined_tif)
    
    build_vrt(str(combined_vrt), tiffs)
    #translate vrt to tif. There is a problem warping with the vrt.
    cmd_translate = f"gdal_translate -of GTiff {combined_vrt} {combined_tif}"
    print(cmd_translate)
    exitcode = WEXITSTATUS(system(cmd_translate))
    if exitcode != 0:
        print("Error in creating a mosaic from selected files\n")
        print(f"\nCommand failed:\n {cmd_translate}")
        assert exitcode == 0
    tiff_path = combined_tif
else:
    pass # do nothing. 

**Reproject tiffs from UTM to EPSG 4326:**

In [ ]:
print("Choose your GDAL compatible precalculated HAND file using the file browser below:")

# prehaps search path where 'hand' is stored and replace Path.cwd()
f = FileChooser(Path.cwd())
display(f)   

In [ ]:
#checking current coordinate reference system

tiff_dir = tiff_path.parent

info = (gdal.Info(str(tiff_path), options = ['-json']))
info = info['coordinateSystem']['wkt']
epsg = info.split('ID')[-1].split(',')[1].replace(']', '')
print(f"EPSG code for Water Extent: {epsg}")

hand_dem = Path(f.selected)
info_hand = (gdal.Info(str(hand_dem), options = ['-json']))
info_hand = info_hand['coordinateSystem']['wkt']
epsg_hand = info_hand.split('ID')[-1].split(',')[1].replace(']', '')
print(f'EPSG for HAND: {epsg_hand}')

# #Reprojecting coordinate system
filename = tiff_path.name
filenoext = Path(filename).stem #given vrt we want to force geotif output with tif extension
from os import symlink
if epsg != epsg_hand:
    cmd_reproj=f"gdalwarp -overwrite -t_srs EPSG:{epsg_hand} -r cubicspline -of GTiff {tiff_dir}/{filename} {tiff_dir}/reproj_{filenoext}.tif"
    print(cmd_reproj)
    system(cmd_reproj)
else:
    if (tiff_dir/f'reproj_{filenoext}.tif').exists():
        (tiff_dir/f'reproj_{filenoext}.tif').unlink()
    symlink(tiff_dir/filename, tiff_dir/f'reproj_{filenoext}.tif')
    
    
# Building the virtual raster for Change Detection product(tiff)
reprojected_flood_mask = tiff_dir/f"reproj_{filenoext}.tif"
print(f"Reprojected Flood Mask File: {reprojected_flood_mask}")

pixels, lines = get_size(str(reprojected_flood_mask))
print(f"X-dimension: {pixels} Y-dimension: {lines}")

In [ ]:
#checking extent of the map
info = (gdal.Info(str(reprojected_flood_mask), options = ['-json']))
west, east, south, north = get_wesn(str(reprojected_flood_mask))
print(f"Retrieved Extent of Flood Extent (w/e/s/n):{west}, {east}, {south}, {north}")

In [ ]:
#Check if HAND is valid. 
hand_dem_bb = bounding_box(str(hand_dem))
if not bounding_box_inside_bounding_box(bounding_box(str(reprojected_flood_mask)), hand_dem_bb):
    print('Flood Extent Bounding Box:')
    print(bounding_box(str(reprojected_flood_mask)))
    print('HAND boundbing box:')
    print(hand_dem_bb)
    print('You can use BIG HAND Notebook to calculate HAND from a DEM.')
    print('Image is not completely covered inside given HAND.')
    print('If you continue your result may not be valid...')
    if yesno("Do you want to continue?"):
        pass
    else:
        raise ValueError('Image is not completely covered inside given HAND.')

#Clip HAND to the same size as the reprojected_flood_mask
filename = hand_dem.name
cmd_clip = f"gdalwarp -overwrite -te {west} {south} {east} {north} -ts {pixels} {lines} -r lanczos  -of GTiff {hand_dem} {tiff_dir}/clip_{filename}"
print(cmd_clip)
system(cmd_clip)

# NO ATTRIBUTE NAMED 'readData'
hand_array = gis.readData(f"{tiff_dir}/clip_{filename}")
if np.all(hand_array==0):
    print('HAND is all zeros. HAND DEM does not cover the imaged area.')
    raise ValueError # THIS SHOULD NEVER HAPPEN now that we are checking the bounding box. Unless HAND is bad.     


# Generating Flood Mask

## Pull Known Perennial Water Information from Public Repository

All perennial Global Surface Water data is produced under the Copernicus Programme: Jean-Francois Pekel, Andrew Cottam, Noel Gorelick, Alan S. Belward, High-resolution mapping of global surface water and its long-term changes. Nature 540, 418-422 (2016). (doi:10.1038/nature20584). **We pull this layer to make sure all perennial water is accounted for in the surface water information that is used for Flood Depth Map calculation.**

In [ ]:
#Get known Water Mask
ths = known_water_threshold #30 #higher than 30% possibility 
known_water_mask = get_waterbody(str(reprojected_flood_mask), ths)
if show_plots:
    pl.matshow(known_water_mask)

## Grabbing Surface Water Extent Map Created in Lab 2

Now we grab the Surface Water Extent Map that we created in Lab 2.

In [ ]:
#load and display change detection product from Hyp3
hyp_map = gdal.Open(str(reprojected_flood_mask))
change_map = hyp_map.ReadAsArray()

#Initial mask layer generation
for c in water_classes: # This allows more than a single water_class to be included in flood mask
    change_map[change_map==c] = 1

mask = change_map == 1
flood_mask = np.bitwise_or(mask,known_water_mask) #add known water mask... #Added 20200921

if show_plots:
    pl.matshow(flood_mask)
    pl.title('Final Flood Mask')

## Flood Depth Map Calculation

Now we **add known water information to the SAR-derived surface water detection maps** and then we **generate our desired Flood Depth Product:**

In [ ]:
import warnings

# Calculate Flood Depth - Show Progress Bar
flood_mask_labels, num_labels = ndimage.label(flood_mask)
print(f'Detected {num_labels} water bodies...')
object_slices = ndimage.find_objects(flood_mask_labels)
if show_plots:
    pl.matshow(flood_mask_labels)
    pl.colorbar()

flood_depth=np.zeros(flood_mask.shape)
print(f'Using estimator: {estimator}')
for l in tqdm(range(1, num_labels)):#Skip first, largest label.
    slices = object_slices[l-1] #osl label=1 is in object_slices[0]
    min0 = slices[0].start
    max0 = slices[0].stop
    min1 = slices[1].start
    max1 = slices[1].stop
    flood_mask_labels_clip = flood_mask_labels[min0: max0, min1: max1] 

    flood_mask_clip = flood_mask[min0: max0, min1: max1].copy()
    flood_mask_clip[flood_mask_labels_clip != l] = 0 #Maskout other flooded areas (labels)
    hand_clip = hand_array[min0: max0, min1: max1] 

# Supressed warning in regards to /tmp/ipykernel_296/3146410417.py:26: RuntimeWarning: Mean of empty slicem=np.nanmean(hand_clip[flood_mask_labels_clip==l])
# If this part crashes, try removing 'warning' 
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', r'Mean of empty slice')
        if estimator.lower() == "numpy":
            m = np.nanmean(hand_clip[flood_mask_labels_clip == l])
            s = np.nanstd( hand_clip[flood_mask_labels_clip == l])
            water_height = m + water_level_sigma * s
        elif estimator.lower() == "nmad":
            m = np.nanmean(hand_clip[flood_mask_labels_clip==l])
            s = stats.median_abs_deviation(hand_clip[flood_mask_labels_clip==l], scale='normal', nan_policy='omit')
            water_height = m + water_level_sigma * s
        elif estimator.lower() == "logstat":
            m = logstat(hand_clip[flood_mask_labels_clip==l], func=np.nanmean)
            s = logstat(hand_clip[flood_mask_labels_clip==l])  
            water_height = m + water_level_sigma * s
        elif estimator.lower() == "iterative":
            water_height = iterative(hand_clip, flood_mask_labels_clip==l, water_levels=iterative_bounds)            
        else:
            print("Unknown estimator selected for water height calculation.")
            raise ValueError

        
    #if np.isnan(m) or np.isnan(s):
    #    set_trace()
    flood_depth_clip = flood_depth[min0:max0, min1:max1]
    flood_depth_clip[flood_mask_labels_clip==l] = water_height - hand_clip[flood_mask_labels_clip==l]    

#remove negative depths:
flood_depth[flood_depth<0] = 0
if show_plots:
    m = np.nanmean(flood_depth)
    s = np.nanstd(flood_depth)
    clim_min = max([m-2*s, 0])
    clim_max = min([m+2*s, 5])
    pl.matshow(flood_depth)
    pl.colorbar()
    pl.clim([clim_min, clim_max])
    pl.title('Estimated Flood Depth')

## Export Your Flood Depth Map as GeoTIFF

In [ ]:
#Saving Estimated FD to geotiff
geotiff_path = work_path/'geotiff'

if not geotiff_path.exists():
    geotiff_path.mkdir()

gT = gis.getGeoTransform(f"{tiff_dir}/clip_{filename}")

outfilename = str(tiff_path).split(str(tiff_dir))[1].split("/")[1]
srs_proj4 = gdal_get_projection(f"{tiff_dir}/clip_{filename}")
gis.writeTiff(flood_depth, gT, filename = "_".join(filter(None, [output_prefix, f"{geotiff_path}/HAND_WaterDepth", estimator, version, outfilename])), srs_proj4=srs_proj4, nodata=0, options = ["TILED=YES","COMPRESS=LZW","INTERLEAVE=BAND","BIGTIFF=YES"])
gis.writeTiff(flood_mask, gT, filename = "_".join(filter(None, [output_prefix, f"{geotiff_path}/Flood_mask", estimator, version, outfilename])), srs_proj4=srs_proj4, options = ["TILED=YES","COMPRESS=LZW","INTERLEAVE=BAND","BIGTIFF=YES"])

flood_mask[known_water_mask] = 0
flood_depth[np.bitwise_not(flood_mask)] = 0
gis.writeTiff(flood_depth, gT, filename = "_".join(filter(None, [output_prefix, f"{geotiff_path}/HAND_FloodDepth", estimator, version, outfilename])), nodata=0, srs_proj4=srs_proj4, options = ["TILED=YES","COMPRESS=LZW","INTERLEAVE=BAND","BIGTIFF=YES"])
print('Export complete.')

## Clean Up Temporary and Intermediate Files

In [ ]:
#clear some intermediate files
try:
    reprojected_flood_mask.unlink()
except:
    pass
try:    
    (tiff_dir/f'clip_{filename}').unlink()
except:
    pass
try:
    (tiff_dir/f'reproj_{filenoext}.tif').unlink()
except:
    pass
try:
    (tiff_dir/f'surface_water_map_clip.tif').unlink()
except:
    pass

---
- *Version 0.1.9 - Batu Osmanoglu, MinJeong Jo*
- *Version 0.1.10 - Rui Kawahara*
- *Version 0.1.11 - Alex Lewandowski*

---

***Change Log***
- *2021/11/23: v0.1.11 - Alex Lewandowski*
    - *Feat: Use url-widget to access url via js, needed for jupyterLab 
    - *Feat: Use ipyfilechooser to shorten notebook*
    - *Feat: Change html to Markdown for better rendering on GitHub*
- *2021/11/8: v0.1.10 - Rui Kawahara* 
    - *Configured in such way that the new Geotiff and S_WATER directory gets generated in source directory (e.g. `BangladeshFloodMapping`).*
    - *Most of `os` modules are updated to `pathlib` counterparts.*
    - *Works with multiple images as well as single image processing.*
- *2021/01/24: v0.1.9 - Batu Osmanoglu, MinJeong Jo*
    - *Added `iterative` estimator. This method is based on `scipy.optimize.basinhopping` with bounds, which can be specified with the `iterative_bounds` parameter. It takes considerably longer to use iterative method as it tried to match the observed flood-extent pattern at different water levels.*
- *2021/01/19:*
    - Minor cleanup and threshold implementation to `get_waterbody`. Also changed the dataset to 2019 (`downloads2019v2`).
- *2020/12/01:*
    - *Added new statistical estimators for maximum water height: numpy, nmad or logstat. Numpy uses standard mean, and std.dev. functions. NMAD uses normalized mean absolute difference for sigma. See `scipy.stats.median_abs_deviation()` for details. logstat uses standard mean and std.dev functions after taking the logarithm of the data. See `logstat()` for details.*
- *2020/11/09:*
    - *Changed known water source, Occurrence 2019 vertion. Added a threshold variable.*
- *2020/10/22:*
    - *BugFix: During reproj EPSG code was wrongly identified. Also if EPSG code is read-wrong the -s_srs flag in gdal_warp causing the reprojection to fail. Fixed both.*
    - *Testing: Replaced standard deviation with normalized mean absolute deviation. For large riverine floods, large water objects result in overestimation of sigma resulting in deeper than expected water depths.*
    - *Feat: Removing negative water depths in the final map.*
- *2020/10/10:*
    - *BugFix: Looks like with the recent updates `gdal.Info(tiff_path, options = ['-json'])` returns a dict instead of a string. Fixed the collection of projection based on this.*
    - *Feat: Allowing to continue even if HAND is smaller than image. This is useful if SAR image covers significant amount of ocean etc.*
    - *BugFix: gis.readData() was failing to read the VRT generated in get_waterbody. gdalwarp outputs a GeoTif now.*
- *2020/10/01:*
    - *Feat: Moving away from repetitive argwhere to ndimage.find_objects results in ~6000x faster calculation.*
- *2020/09/30:*
    - *BugFix: The known water mask and input mask was merged using a sum operator instead of a bitwise_or operator.*
- *2020/09/20:*
    - *BugFix: Added known water download and addition to the mask. This helps to make sure known water bodies are handled as a single object. Removed morphological filters also.*
- *2020/09/17:*
    - *First version.
- *2021/04/016:*
    - *Update: import gdal and osr from osgeo*